In [1]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections, copy, pickle
from termcolor import colored
from importlib import reload
from dateutil.parser import parse
import scipy.linalg, scipy.stats
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
import seaborn as sns
from IPython.display import HTML

In [78]:
import scipy.stats
from sklearn import preprocessing
import keras
import keras.utils
from keras import optimizers, backend as K
from keras.layers import *
from keras.models import Model

In [165]:
import util.plot
import util.data

In [166]:
reload(util.plot)
reload(util.data);

In [167]:
# with open('data/encoder.pkl', 'rb') as f:
#     E = pickle.load(f)

In [168]:
data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=1*1000)
data_test = pd.read_csv('data/test_set_VU_DM.csv', sep=',', nrows=1000)
data.shape, data.srch_id.unique().size

((1000, 79), 40)

In [169]:
data.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,minute,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,score,travel_distance
0,1,0.0,8.0,0.098734,0.006029,0.0,893,-0.190084,-0.300346,1,...,32,0,0,0,0,1,0,0,0,NaN
1,1,0.0,8.0,0.098734,0.006029,0.0,10404,0.763816,0.191146,1,...,32,0,0,0,0,1,0,0,0,NaN
2,1,0.0,8.0,0.098734,0.006029,0.0,21315,-0.190084,0.682638,1,...,32,0,0,0,0,1,0,0,0,NaN
3,1,0.0,8.0,0.098734,0.006029,0.0,27348,-1.143983,0.191146,1,...,32,0,0,0,0,1,0,0,0,NaN
4,1,0.0,8.0,0.098734,0.006029,0.0,29604,0.763816,-0.300346,1,...,32,0,0,0,0,1,0,0,0,NaN


# AE 1) Search id

See also https://blog.keras.io/building-autoencoders-in-keras.html and https://github.com/voschezang/drum-style-transfer

In [170]:
# [k for k in data.columns if 'comp' not in k and 'prop' not in k]

In [171]:
# rm irrelevant info
keys_property = [k for k in data.columns if 'comp' in k or 'prop' in k]
keys_property += ['click_bool', 'booking_bool', 'gross_bookings_usd', 'random_bool', 'score', 'price_usd', 'position', 'travel_distance', 'travel_distances']
keys_search = [k for k in data.columns if k not in keys_property]
len(keys_search), keys_search

(36,
 ['srch_id',
  'site_id',
  'visitor_location_country_id',
  'visitor_hist_starrating',
  'visitor_hist_adr_usd',
  'promotion_flag',
  'srch_destination_id',
  'srch_length_of_stay',
  'srch_booking_window',
  'srch_adults_count',
  'srch_children_count',
  'srch_room_count',
  'srch_saturday_night_bool',
  'srch_query_affinity_score',
  'orig_destination_distance',
  'visitor_hist_starrating_is_null',
  'srch_person_per_room_score',
  'srch_adults_per_room_score',
  'srch_adults_count_float',
  'srch_children_count_float',
  'srch_room_count_float',
  'position_float',
  'srch_length_of_stay_float',
  'srch_booking_window_float',
  'year',
  'month',
  'day',
  'hour',
  'minute',
  'Friday',
  'Monday',
  'Saturday',
  'Sunday',
  'Thursday',
  'Tuesday',
  'Wednesday'])

In [172]:
data = data[keys_search]
data.srch_id.unique()
data_unique_srch_id = data.drop_duplicates(subset='srch_id')
assert data_unique_srch_id.shape[0] == data.srch_id.unique().size, \
    'if srch_id is equal, all non-property attributes should be equal as well'

In [206]:
assert data.srch_id.isna().sum() == 0
assert data.srch_id.isin([np.nan, np.inf, -np.inf]).sum() == 0
assert data.srch_id.min() > 0
# assert data.srch_id.isna()

In [188]:
# convert to np array & reshape
data = data.sample(frac=1, random_state=seed)
x_train = data_unique_srch_id.values[:,:,np.newaxis]
y_labels = x_train[:,0,:]
discretizer = preprocessing.KBinsDiscretizer(n_bins=y_labels.shape[0],
         encode='onehot', strategy='uniform')
y_train = discretizer.fit_transform(y_labels)
y_labels.shape, y_train.shape

((40, 1), (40, 40))

In [189]:
y_train = y_train.toarray()
y_train = y_train[:,:,np.newaxis]
y_train.shape

(40, 40, 1)

In [198]:
# n_dims = data.shape[1]
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]
print(input_shape, output_shape)
encoder_input = Input(shape=input_shape)
h = encoder_input
h = Flatten()(h)
h = Dense(100, activation='relu')(h)
# decoder
h = Dense(np.prod(output_shape), activation='relu')(h)
h = Reshape(output_shape)(h)
encoder_output = Activation('sigmoid')(h)
model = Model(encoder_input, encoder_output, name='ae-')
model.compile(optimizer='adam', loss='binary_crossentropy') # RMSprop Adadelta SGD

(36, 1) (40, 1)


In [197]:
# validation_split does not shuffle the data
result = model.fit(x_train, y_train, epochs=2, batch_size=int(x_train.shape[0]/2), validation_split=0.2)
# history = result.history

Train on 32 samples, validate on 8 samples
Epoch 1/2
32/32 [==============================] - 1s 34ms/step - loss: nan - val_loss: nan
Epoch 2/2
32/32 [==============================] - 0s 224us/step - loss: nan - val_loss: nan


In [160]:
# result.params
result.history

{'loss': [nan, nan]}

In [161]:
model.predict(x_train)

array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[1. ],
        [0.5],
        [0.5],
        ...,
        [1. ],
        [1. ],
        [1. ]],

       [[1. ],
        [0.5],
        [0.5],
        ...,
        [1. ],
        [1. ],
        [1. ]],

       ...,

       [[1. ],
        [0.5],
        [0.5],
        ...,
        [1. ],
        [1. ],
        [1. ]],

       [[1. ],
        [0.5],
        [0.5],
        ...,
        [1. ],
        [1. ],
        [1. ]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]], dtype=float32)

In [ ]:
def init_encoder(input_shape=(10,1), latent_dim=2):
    encoder_input = Input(shape=input_shape)
    n_nodes = np.prod(input_shape)

    # Convolution
    h = encoder_input
    h = Reshape((timesteps, notes))(h)
    h = Conv1D(
        64, kernel_size=2, strides=1, activation='relu', padding='valid')(h)

    h = Bidirectional(LSTM(128))(h)

    # Z Mean, Variance
    z_mean = Dense(latent_dim, name='z_mean')(h)  # no activation='relu'
    z_log_var = Dense(latent_dim, name='z_log_var')(h)  # no activation='relu'

    encoder_output = [z_mean, z_log_var]
    encoder_model = Model(encoder_input, encoder_output, name='encoder_model-')
    return encoder_model, encoder_input, z_mean, z_log_var


def init(input_shape=(36, 1), latent_dim=10, epsilon_std=1.):
    encoder_model, encoder_input, z_mean, z_log_var = encoder(
        input_shape, latent_dim)
    z_input = encoder_model(encoder_input)

    # this function must be defined locally in order for the model to be serializable
    # using the following inline lambda function will result in an error when .to_json() is called
    #   lambda args: sample(args, z_mean, z_log_var, latent_dim, epsilon_std)
    def sample(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(
            shape=(K.shape(z_mean)[0], latent_dim),
            mean=0.,
            stddev=epsilon_std)
        return z_mean + K.exp(z_log_var) * epsilon

    z_output = Lambda(sample)(z_input)
    decoder_model = decoder(z_output, latent_dim, input_shape)

    # VAE
    vae_input = Input(shape=(input_shape, ))
    vae_input = encoder_input
    vae_output = decoder_model(z_output)
    # vae_output = decoder_model(z_mean)
    vae = Model(vae_input, vae_output, name='vae-model-')

    vae_loss_ = vae_loss(
        vae_input,
        vae_output,
        z_mean,
        z_log_var,
        timesteps,
        notes,
        beta=0.75,
        gamma=0.05)
    vae.add_loss(vae_loss_)
    vae.compile(optimizer='adam')

    # Encoder (matrices -> latent vectors)
    encoder_ = Model(encoder_input, z_mean)
    # Generator (latent vectors -> matrices)
    # generator_input = Input((latent_dim, ))
    # generator_layers_ = utils.composition(decoders, generator_input)
    # generator = Model(generator_input, generator_layers_)
    return vae, encoder_, decoder_model
